# Imports

In [1]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

In [2]:
def get_defining_set(n, pol, field):
    """
    Return a tuple of ``n`` distinct elements of ``field``
    that are not roots of ``pol`` 
    
    INPUT:
        - ``n`` -- length
        - ``pol`` -- a monic polynomial with coefficients in `field`
        - ``field`` -- finite field
    """
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set

In [3]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`
    - ``defining_set`` -- tuple of `n` distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()

        if not self._field.is_field() or not self._field.is_finite():
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
        if self.get_dimension() == 0:
            raise ValueError("ERROR. Code dimension is null")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())
    
class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        self._generator_matrix = self.code().get_generator_matrix()
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self._generator_matrix
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m`` -- a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c`` -- a element of the input space of ``self``
        """   
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):
            syndrome = syndrome + c[i]*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword
        
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 0

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t or r_prev.degree() < t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
        
        # Step 5
        # make sigma monic     
        sigma = U_i / U_i.coefficients()[-1]
        
        eta = (-1)^i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        for root in sigma.roots():
            if g(root[0] != 0):
                roots_loc = roots_loc + [self._defining_set.index(root[0])]
        
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(roots_loc)):
            error[roots_loc[i]] = eta.subs(x = sigma.roots()[i][0]) / (sigma_diff.subs(x = sigma.roots()[i][0]))
        
        x = word - vector(self.code()._field, error)
        
        return x

    def decode_to_message(self, word):
        r"""
        Decode ``word`` to the message space.
        
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        word = self.decode_to_code(word)
        message = self.code().get_generator_matrix().solve_left(word)
        return message

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

In [4]:
def get_weight(c):
    """
    Return the weight of ``c``
    
    INPUT:
        - ``c`` -- a vector
    """
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [5]:
class McEliece:
    r"""
    Implementation of McEliece cryptosystem.
    
    INPUT:
    - ``n`` -- length
    - ``q`` -- Dimension of the finite field `GF(2^q)`
    - ``g``-- a monic polynomial with coefficients in
    a finite field `\GF{2^q}`
    """
    def __init__(self, n, q, g):
        """
        Initialize.
        """
        # Goppa code
        F = GF(2)
        L = GF(2^q)
        defining_set = get_defining_set(n, g, L)
        C = Goppa(defining_set, g, F)
        G = C.get_generator_matrix()
        k = G.nrows()
        n = G.ncols()
    
        # random binary nonsingular matrix
        S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
        while rank(S) < k:
            i = randint(0, k-1)
            j = randint(0, k-1)
            S[i,j] = choice(F.list())
        
        # random permutation matrix
        columns = list(range(n))
        P = matrix(F, n)
        
        for i in range(n):
            l = randint(0, len(columns)-1)
            j = columns[l]
            P[i,j] = 1
            columns.remove(j)
        
        self._goppa = C
        self._G = G
        self._t = floor(g.degree()/2)
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._public_key = S * self._G * P
        
    def __repr__(self):
        """
        Representation of a McEliece cryptosystem.
        """
        return "McEliece cryptosystem over {}".format(self._goppa)
        
    def get_S(self):
        """
        Return the random binary nonsingular matrix that is part of the private key
        """
        return self._S
    
    def get_G(self):
        """
        Return the generating matrix associated with the Goppa code that is part of the private key
        """
        return self._G
    
    def get_P(self):
        """
        Return the random permutation matrix that is part of the private key
        """
        return self._P
    
    def get_public_key(self):
        """
        Return the public key
        """
        return self._public_key

    def encrypt(self, m):
        """
        Return a chipertext
        
        INPUT:
        - ``m`` -- a plaintext to encrypt
        """
        # random errors
        e = vector([F(0) for i in range(self._n)])
        
        while get_weight(e) != self._t:
            i = randint(0, len(e) - 1)
            e[i] = choice(F.list())
        
        # chipertext
        c = m * self._public_key + e
        
        return c
    
    def decrypt(self, c):
        """
        Return the plain text associated with the cryptogram ``c``
        
        INPUT:
        - ``c`` -- a chiphertext
        """
        word = c * self._P^(-1)
        D = GoppaDecoder(self._goppa)
        word = D.decode_to_message(word)
        message = word * (self._S)^(-1)
        
        return message

# GGA

In [6]:
def fitness_vector(v):
    """
    Return the fitness of the vector ``v``
    
    INPUT:
        - ``v`` -- a binary vector
    """
    f = 0
    
    for i in v:
        if i != 0:
            f += 1
    
    return f

In [7]:
def fitness_matrix(M):
    """
    Return the minimum fitness of the rows of ``M``
    
    INPUT:
        - ``M`` -- a binary matrix
    """
    min_f = Infinity
    
    for row in M:
        f = fitness_vector(row)
        
        if f < min_f:
            min_f = f
    
    return min_f

In [8]:
def permutation_matrix(x):
    """
    Return the matrix of permutations associated with ``x``
    
    INPUT:
        - ``x`` -- a permutation
    """
    M = matrix(GF(2), len(x), len(x))
    
    for j in range(len(x)):
        i = x[j] - 1
        M[i,j] = 1
    
    return M

In [9]:
def fitness_permutation(M, x):
    """
    Return the the minimum fitness of the rows of the 
    reduced row echelon form of the matrix ``M`` 
    permuted by ``x``
    
    INPUT:
        - ``M`` -- a binary matrix
        - ``x`` -- a permutation applied to ``M``
    """
    P = permutation_matrix(x)
    
    return fitness_matrix((M * P).rref())

In [10]:
def random_sol(n):
    """
    Return a random vector of length ``n``
    
    INPUT:
        - ``n`` -- length
    """
    sol = []
    
    while len(sol) != n:
        r = randint(1, n)
        
        if r not in sol:
            sol = sol + [r]
            
    return sol

In [11]:
def crossover(p1, p2):
    """
    Return the descendant of the cross of ``p1`` and ``p2``
    
    INPUT:
        - ``p1`` -- a vector of length ``n``
        - ``p2`` -- a vector of length ``n``
    """
    return Permutation(p1) * Permutation(p2), Permutation(p2) * Permutation(p1)

In [12]:
def mutation(p):
    """
    Return a mutated vector
    
    INPUT:
        - ``p`` -- a vector
    """
    n = len(p)
    k = randint(1, n - 1)
    k1 = randint(0, k - 1)
    k2 = randint(k, n - 1)
    
    p[k1], p[k2] = p[k2], p[k1]

    return p

In [13]:
def GGA(G, N, pc, max_reinit, min_fitness):
    """
    Return a permutation of the ``G`` whose reduced row echelon form
    has a row with weight ``min_fitness``
    
    INPUT:
        - ``G`` -- a matrix made up of the public key of a McEliece 
          cryptosystem and a row with the encrypted message
        - ``N`` -- population size
        - ``pc`` -- crossover probability
        - ``max_reinit`` -- maximum number of evaluations of the 
          solutions that do not produce improvement in the fitness 
          of the best solution found.
        - ``min_fitness`` -- minimun fitness
    """
    t = 0
    max_t = 500000
    Pt = []
    reinit = 0
    n = G.ncols()
    
    
    # Initialize the Population P(t)
    for i in range(0, N):
        sol = random_sol(n)
        Pt = Pt + [sol]
    
    Pt = Matrix(Pt)
    
    # Evaluate
    fitness_Pt = []
    for Pti in Pt:
        fitness_Pt = fitness_Pt + [fitness_permutation(G, Pti)]
        
    while(t < max_t and min(fitness_Pt) > min_fitness):
        Pt_sig = []
        fitness_Pt_sig = []
        
        # Binary tournament selection
        parents = []
        for i in range(0, N):            
            ind1 = Pt[randint(0, N-1)]
            ind2 = Pt[randint(0, N-1)]
            
            if (fitness_permutation(G, ind1) >= fitness_permutation(G, ind2)):
                parents = parents + [ind2]
            else:
                parents = parents + [ind1]
        
        parents = Matrix(parents)
        
        # Generation
        n_crossovers = pc * N/2 # number of crossovers

        for i in range(0, N/2):            
            if n_crossovers > 0:
                c1, c2 = crossover(parents[2*i], parents[2*i + 1])
                n_crossovers -= 1
            else:
                c1 = mutation(copy(parents[2*i]))
                c2 = mutation(copy(parents[2*i + 1]))
            
            # Update
            Pt_sig = Pt_sig + [c1]
            Pt_sig = Pt_sig + [c2]
        
        Pt_sig = Matrix(Pt_sig)
        
        # Evaluate
        for Pt_sigi in Pt_sig:
            fitness_Pt_sig = fitness_Pt_sig + [fitness_permutation(G, Pt_sigi)]
        
        # No improvement
        if min(fitness_Pt_sig) > min(fitness_Pt):
            worst = fitness_Pt_sig.index(max(fitness_Pt_sig))
            best = fitness_Pt.index(min(fitness_Pt))
            Pt_sig[worst] = Pt[best]
            fitness_Pt_sig[worst] = fitness_Pt[best]
            reinit += 1
        else:
            reinit = 0
        
        # Restart
        if reinit >= max_reinit:
            Pt_sig = []
            for i in range(0, N-1):
                Pt_sig = Pt_sig + [random_sol(n)]
            
            best = fitness_Pt.index(min(fitness_Pt))
            Pt_sig = Pt_sig + [Pt[best]]
            Pt_sig = Matrix(Pt_sig)
            
            # Evaluate
            fitness_Pt_sig = []
            for Pt_sigi in Pt_sig:
                fitness_Pt_sig = fitness_Pt_sig + [fitness_permutation(G, Pt_sigi)]
        
        fitness_Pt = fitness_Pt_sig
        Pt = Pt_sig
        t += 1
    
    best = fitness_Pt.index(min(fitness_Pt))
    
    return Pt[best], fitness_permutation(G, Pt[best])

# CHC

In [14]:
def distance(x, y):
    """
    Return de Hamming distance between ``x`` and ``y``
    
    INPUT:
        - ``x`` -- a vector of length ``n``
        - ``y`` -- a vector of length ``n``
    """
    assert(len(x) == len(y))
    
    n = len(x)
    count = 0
    for i in range(0, n):
        if x[i] != y[i]:
            count += 1
            
    return count

In [15]:
def actualizar(P, tau):
    """
    Update the crossover decrement
    
    INPUT:
        - ``P`` -- population
        - ``tau`` -- crossover rate
    """
    d = 0
    max_dist = 0
    count = 0
    
    for i in range(0, P.nrows()):
        for j in range(i, P.nrows()):
            dist = distance(P[i], P[j])
            
            if dist > max_dist:
                max_dist = dist
            
            d += dist
            count += 1
    
    d = d * 1.0/count
    dec = tau * max_dist
    
    return d, dec

In [16]:
def CHC(G, N, tau, min_fitness):
    """
    Return a permutation of the ``G`` whose reduced row echelon form
    has a row with weight ``min_fitness``
    
    INPUT:
        - ``G`` -- a matrix made up of the public key of a McEliece 
          cryptosystem and a row with the encrypted message
        - ``N`` -- population size
        - ``tau`` -- crossover rating
        - ``min_fitness`` -- minimun fitness
    """
    t = 0
    max_t = 10000
    Pt = []
    reinit = 0
    n = G.ncols()
    
    # Initialize the Population P(t)
    for i in range(0, N):
        sol = random_sol(n)    
        Pt = Pt + [sol]
    
    Pt = Matrix(Pt)
    
    # Evaluate
    fitness_Pt = []   
    for Pti in Pt:
        fitness_Pt = fitness_Pt + [fitness_permutation(G, Pti)]
    
    # Distance
    d, dec = actualizar(Pt, tau)
        
    while(t < max_t and min(fitness_Pt) > min_fitness):
        Ct = []
        Pt_sig = []
        fitness_Ct = []
        fitness_Pt_sig = []
        improvement = False
        
        # Parent selection
        parents = []
        for i in range(0, N):            
            p = randint(0, N-1)
            parents = parents + [Pt[p]]
        
        parents = Matrix(parents)
        
        # Generation
        for i in range(0, N/2):            
            if distance(parents[2*i], parents[2*i + 1]) < d:
                c1, c2 = crossover(parents[2*i], parents[2*i + 1])
                Ct = Ct + [c1]
                Ct = Ct + [c2]
        
        Ct = Matrix(Ct)
        
        # Evaluate
        for Cti in Ct:
            fitness_Ct = fitness_Ct + [fitness_permutation(G, Cti)]
        
        if Ct.nrows() == 0:
            Pt_sig = Pt
            fitness_Pt_sig = fitness_Pt
        else:
            fitness_Pt_aux = fitness_Pt
            fitness_Ct_aux = fitness_Ct
            
            for i in range(0, N):
                best_Pt = min(fitness_Pt_aux)
                best_Ct = min(fitness_Ct_aux)

                if best_Pt < best_Ct:
                    index = fitness_Pt_aux.index(best_Pt)
                    Pt_sig = Pt_sig + [Pt[index]]
                    fitness_Pt_sig = fitness_Pt_sig + [best_Pt]
                    fitness_Pt_aux[index] = Infinity
                else:
                    index = fitness_Ct_aux.index(best_Ct)
                    Pt_sig = Pt_sig + [Ct[index]]
                    fitness_Pt_sig = fitness_Pt_sig + [best_Ct]
                    fitness_Ct_aux[index] = Infinity
                    improvement = True
        
            Pt_sig = Matrix(Pt_sig)
        
        # No improvement
        if not improvement:
            d = d - dec
            
            if d <= 0:
                Pt_sig = []
                for i in range(0, N-1):
                    Pt_sig = Pt_sig + [random_sol(n)]

                best = fitness_Pt.index(min(fitness_Pt))
                Pt_sig = Pt_sig + [Pt[best]]
                Pt_sig = Matrix(Pt_sig)
                
                # Distance
                d, dec = actualizar(Pt, tau)
        
        fitness_Pt = fitness_Pt_sig
        Pt = Pt_sig
        t += 1
    
    best = fitness_Pt.index(min(fitness_Pt))
    
    return Pt[best], fitness_permutation(G, Pt[best])

# EJEMPLOS

### Ejecutar un bloque de los siguientes para elegir el tamaño $n$.

### n = 64

In [17]:
q = 8
n = 64
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]
g = x^4 + (a + 1)*x^3 + (a^6 + a^2 + a)*x^2 + (a^6 + a)*x + a^2 + a + 1
t = floor(g.degree()/2)

### n = 128

In [18]:
q = 8
n = 128
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]
g = x^8 + a*x^7 + (a^2 + a + 1)*x^4 + x + 1
t = floor(g.degree()/2)

### n = 256

In [19]:
q = 9
n = 256
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]
g = x^14 + (a^8 + a^5 + a^3)*x^10 + x^2 + a + 1
t = floor(g.degree()/2)

### n = 512

In [20]:
q = 10
n = 512
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]
g = x^25 + a*x^15 + (a^2 + 1)*a*x^10 + a
t = floor(g.degree()/2)

### McEliece

In [21]:
ME = McEliece(n, q, g)
ME

McEliece cryptosystem over [512, 262] Goppa code

In [22]:
PK = ME.get_public_key()
message = []
while len(message) != PK.nrows():
    message = message + [choice(F.list())]

message = vector(F, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = ME.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)


Message to encrypt:
(0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1)

Encrypted message:
(0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 

In [23]:
G = PK.stack(encrypted_message)

### GGA

In [24]:
p, f = GGA(G, 300, 0.7, 100000, t)
P = permutation_matrix(p)
M = (G*P).rref()

# row with min fitness
for row in M:
    if fitness_vector(row) == f:
        v = vector(GF(2), row)
        break


e = v*P^(-1) # error
z = encrypted_message - e # y - e
m = PK.solve_left(z) # z = mPK

print("Mensaje obtenido:")
print(m)
print("\nMensaje original:")
print(message)
print("\nSon iguales:")
print(message == m)

Mensaje obtenido:
(0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1)

Mensaje original:
(0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1

### CHC

In [25]:
p, f = CHC(G, 350, 0.8, t)
P = permutation_matrix(p)
M = (G*P).rref()

# row with min fitness
for row in M:
    if fitness_vector(row) == f:
        v = vector(GF(2), row)
        break


e = v*P^(-1) # error
z = encrypted_message - e # y - e
m = PK.solve_left(z) # z = mPK

print("Mensaje obtenido:")
print(m)
print("\nMensaje original:")
print(message)
print("\nSon iguales:")
print(message == m)

Mensaje obtenido:
(0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1)

Mensaje original:
(0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1

# Temporal

In [26]:
q = 4
n = 14
t = 2
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]
g = x^2 + (a^3 + a^2 + a)*x + a^2 + a + 1
min_fitness = floor(t/2)
ME = McEliece(n, q, g)
G = ME.get_public_key()
message = vector(F, (0, 0, 1, 1, 1, 0))
encrypted_message = ME.encrypt(message)
G = G.stack(encrypted_message)
CHC(G, 400, 0.8, min_fitness)

((5, 8, 9, 2, 6, 3, 12, 10, 1, 13, 7, 14, 4, 11), 1)

In [27]:
fitness_vector([0, 1, 1, 0, 0, 0, 1])

3

In [28]:
fitness_matrix(matrix(GF(2), [[0,1], [1, 1]]))

1

In [29]:
M = matrix(GF(2), [[1, 0, 0], [0, 1, 1], [1, 0, 1]]);
x = Permutation([3,2,1])
permutation_matrix(x)
fitness_permutation(M, x)

1

In [30]:
random_sol(5)

[3, 4, 5, 2, 1]

In [31]:
crossover([1, 3, 2], [3, 1, 2])

([3, 2, 1], [2, 1, 3])

In [32]:
Permutation([1,2], [2,1])

[1, 2]

In [33]:
distance([1, 3, 2], [3, 1, 2])

2

In [34]:
P = matrix([[1, 4, 2, 3], [2, 1, 3, 4]])
tau = 0.8
actualizar(P, tau)

(1.33333333333333, 3.20000000000000)